In [1]:
import time 
import pandas as pd
import re

from datetime import date, timedelta
from selenium import webdriver 
from selenium.webdriver import Chrome 
from selenium.webdriver.common.by import By 

from bs4 import BeautifulSoup as bs

from scraping import *

In [2]:
# Define the Chrome webdriver options
options = webdriver.ChromeOptions() 
options.add_argument('--headless')

In [3]:
# By default, Selenium waits for all resources to download before taking actions.
# However, we don't need it as the page is populated with dynamically generated JavaScript code.
options.page_load_strategy = "none"
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--no-sandbox")
#options.add_argument("--headless")  # Old headless Chrome
# OR:
options.add_argument("--headless=new")  # New headless Chrome

In [4]:
# Pass the defined options objects to initialize the web driver 
driver = Chrome(options=options)

In [5]:
# Set an implicit wait of 5 seconds to allow time for elements to appear before throwing an exception
driver.implicitly_wait(3)

In [6]:

query = "mt-07"
urls = []
N = 9

for i in range(N):
    url = "https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/{:}/#q:{:}|searchInTitleAndDescription:true".format(i, query)
    urls.append(url)
    

In [8]:
listings_per_page = {}

for i, url in enumerate(urls):

    print(i, url)
    
    driver.get(url) 
    time.sleep(5)
    html = driver.page_source

    more_listings = html_to_listings(html)

    listings_per_page[i] = more_listings
    

0 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/0/#q:mt-07|searchInTitleAndDescription:true
1 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/1/#q:mt-07|searchInTitleAndDescription:true
2 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/2/#q:mt-07|searchInTitleAndDescription:true
3 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/3/#q:mt-07|searchInTitleAndDescription:true
4 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/4/#q:mt-07|searchInTitleAndDescription:true
5 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/5/#q:mt-07|searchInTitleAndDescription:true
6 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/6/#q:mt-07|searchInTitleAndDescription:true
KeyError: 'title'
7 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/7/#q:mt-07|searchInTitleAndDescription:true
8 https://www.marktplaats.nl/l/motoren/motoren-yamaha/p/8/#q:mt-07|searchInTitleAndDescription:true


In [14]:
listings = []

for page in listings_per_page:
    listings += listings_per_page[page]
    

In [15]:
import os

dirname = "data"

if not os.path.exists(dirname):
    os.mkdir(dirname)


In [16]:
df = pd.DataFrame.from_dict([l._to_dict() for l in listings])

In [20]:

today = date.today()

mfn = "{:}_db.csv".format(query)
fn = "{:}_{:}.csv".format(query,today)
fp = os.path.join(dirname, fn)
mfp = os.path.join(dirname, mfn)

In [25]:
df.to_csv(fp)

In [23]:
mdf = pd.read_csv(mfp, index_col = 0)

In [38]:

s1 = mdf['hash']
s2 = df['hash']

new_hashes = list(set(s2).difference(set(s1)))

new_rows = df.query('hash in {:}'.format(new_hashes))


In [39]:
new_rows

,build_year,date,hash,im_alt,im_src,im_title,location,mileage,price,scrape_date,text,title
